# Synthetic Data

In [1]:
# import libraries

import warnings
warnings.filterwarnings('ignore')

# Import the numpy and pandas package
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import random
#import matplotlib.gridspec as gridspec
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error
from tensorflow.keras import regularizers

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import optimizers,initializers,regularizers

## Produce Synthetic Data

In [2]:
np.random.seed(0)

x = np.arange(-10*np.pi,10*np.pi,0.01)
y = x+np.random.normal(loc=0,scale=0.1)
u = x+np.random.normal(loc=0,scale=0.1)
v = x+np.random.normal(loc=0,scale=0.1)

theta = np.pi/2
c, s = np.cos(theta), np.sin(theta)
rot_90_x = c*x-s*y
rot_90_y = s*x+c*y
rot_90_u = c*u-s*v
rot_90_v = s*u+c*v

trans_1_x = x+1
trans_1_y = y+1
trans_1_u = u+1
trans_1_v = v+1

rot90_trans1x1y_x = c*trans_1_x-s*trans_1_y
rot90_trans1x1y_y = s*trans_1_x+c*trans_1_y
rot90_trans1u1v_u = c*trans_1_u-s*trans_1_v
rot90_trans1u1v_v = s*trans_1_u+c*trans_1_v

cos_x = np.cos(x)
cos_y = np.cos(y)
cos_u = np.cos(u)
cos_v = np.cos(v)

sin_x = np.sin(x)
sin_y = np.sin(y)
sin_u = np.sin(u)
sin_v = np.sin(v)

x_squared = x**2
y_squared = y**2
u_squared = u**2
v_squared = v**2

In [3]:
LinearData = False # True for linear data, False for linear+nonlinear data

syn_data = np.vstack((x,y,u,v,
                        rot_90_x,rot_90_y,rot_90_u,rot_90_v,
                        trans_1_x,trans_1_y,trans_1_u,trans_1_v,\
                        rot90_trans1x1y_x,rot90_trans1x1y_y,rot90_trans1u1v_u,rot90_trans1u1v_v,\
                        cos_x,cos_y,cos_u,cos_v,\
                        sin_x,sin_y,sin_u,sin_v,\
                        x_squared,y_squared,u_squared,v_squared
                            )).T

if LinearData:
    syn_data = syn_data[:,:-12]

In [4]:
syn_data.shape

(6284, 28)

In [5]:
# Split to training and test 80:20
syn_data_train,syn_data_test = train_test_split(syn_data,test_size=0.2,random_state=42)

In [6]:
syn_data_train.shape

(5027, 28)

In [7]:
#Pre-processing
scaler=StandardScaler()
syn_data_train_scal = scaler.fit_transform(syn_data_train)
syn_data_test_scal = scaler.transform(syn_data_test)

## SDAE functions

In [8]:
# Weighted layer defintion with zero-to-one clipping constraint and 1s initialzation with Lasso regularization
class SelectiveLayer(keras.layers.Layer):
    def __init__(self, lasso_rate=0.001,convergence_rate=0.001,*args, **kwargs):
        super().__init__(*args, **kwargs)
        self.lasso_rate = lasso_rate
        #self.convergence_rate = convergence_rate
    def build(self, input_shape):
        self.kernel = self.add_weight("kernel", shape=(int(input_shape[-1]),), 
                                      initializer = initializers.RandomUniform(minval=0.999999, maxval=0.9999999),
                                      regularizer=regularizers.l1(self.lasso_rate),
                                      constraint = ZeroToOneClip()
                                     )
    def call(self, inputs):
        return tf.multiply(inputs, self.kernel)
    def get_config(self):
        config = super().get_config().copy()
        return config


class ZeroToOneClip(tf.keras.constraints.Constraint):
    def __call__(self, w):
        w_new = tf.clip_by_value(w, 0, 1)
        return w_new



def SelectDAE(input_shape,
                  nbr_hidden_layers =1,
                  hidden_layer_shape=13,
                  encodings_nbr = 6,
                  activation="linear",
                  lasso_rate=0.001,
                  sl_lasso_rate=0.1):
    #Encoder 1:Input
    feature_inputs = Input(shape=[input_shape],name='input')
    #Encoder 2: Input
    selective_layer = SelectiveLayer(lasso_rate=sl_lasso_rate)
    feature_selection_choose = selective_layer(feature_inputs)
    
    for i in range(nbr_hidden_layers):
        #Encoder 1: hidden layers
        encoder_layer_full = Dense(hidden_layer_shape, activation=activation,
                            name='encoder_hidden_layer_full_'+str(i))
        #Encoder 2: hidden layers
        encoder_layer_select = Dense(hidden_layer_shape, activation=activation,
                            kernel_regularizer=regularizers.l1(lasso_rate),
                            bias_regularizer=regularizers.l1(lasso_rate),
                            name='encoder_hidden_layer_select_'+str(i))
        if i==0:
            encoder_layer_full_output = encoder_layer_full(feature_inputs)
            encoder_layer_select_output = encoder_layer_select(feature_selection_choose)
        else:
            encoder_layer_full_output = encoder_layer_full(encoder_layer_full_output)
            encoder_layer_select_output = encoder_layer_select(encoder_layer_select_output)
    
    #Encoder 1: Encodings
    encoding_layer_full = Dense(encodings_nbr, activation=activation,
                           name='encoding_layer_full')
    encoding_layer_full_output = encoding_layer_full(encoder_layer_full_output)
    
    #Encoder 2: Encodings
    encoding_layer_select = Dense(encodings_nbr, activation=activation,
                           kernel_regularizer=regularizers.l1(lasso_rate),
                           bias_regularizer=regularizers.l1(lasso_rate),
                           name='encoding_layer_select')
    encoding_layer_select_output = encoding_layer_select(encoder_layer_select_output)
    
    #Decoder Layers
    for i in range(nbr_hidden_layers):
        decoder_layer = Dense(hidden_layer_shape, activation=activation,
                            name='decoder_hidden_layer_'+str(i))
        if i==0:
            decoder_layer_full_output = decoder_layer(encoding_layer_full_output)
            decoder_layer_select_output = decoder_layer(encoding_layer_select_output)
        else:
            decoder_layer_full_output = decoder_layer(decoder_layer_full_output)
            decoder_layer_select_output = decoder_layer(decoder_layer_select_output)
    
    #Reconstruction Layer
    reconstruction_layer = Dense(input_shape, activation='linear',
                                 name='reconstruction_layer')
    recons_layer_full_output = reconstruction_layer(decoder_layer_full_output)
    recons_layer_select_output = reconstruction_layer(decoder_layer_select_output)

    latent_encoder_full = Model(feature_inputs, encoding_layer_full_output,name='fullFeats_Encoder')
    latent_encoder_select = Model(feature_inputs, encoding_layer_select_output,name='SelectFeats_Encoder')
    feature_selection_output=Model(feature_inputs,feature_selection_choose,name='SelectFeats_Layer')
    autoencoder = Model(feature_inputs, recons_layer_full_output,name='DAE')
    autoencoder_select = Model(feature_inputs, recons_layer_select_output,name='SelectDAE')   
    
    print('Autoencoder Structure-------------------------------------')
    autoencoder.summary()
    
    print('SelectEncoder Structure-------------------------------------')
    latent_encoder_select.summary()
    
    return autoencoder,autoencoder_select,feature_selection_output, latent_encoder_full,latent_encoder_select

## Build Model

In [9]:
#parameters
lasso_rate = 1e-5
hidden_layer_shape = 12
nbr_hidden_layers =3
encodings_nbr = 6
if LinearData:
    lasso_rate_SL=2e-3
else:
    lasso_rate_SL=1e-2
activation = "linear"
learning_rate = 1e-3
nbr_batches = 20
batch_size = int(np.floor(syn_data_train.shape[0]/nbr_batches))
seed = 0


random.seed(seed)
rndm_seed = random.randint(1,10000)
tf.random.set_seed(rndm_seed)

DAE,\
Select_DAE,\
SelectLayer_output,\
FullFeats_Encoder,\
SelectFeats_Encoder=SelectDAE(input_shape=syn_data_train.shape[1],
                              nbr_hidden_layers = nbr_hidden_layers,
                              hidden_layer_shape=hidden_layer_shape,
                              encodings_nbr =encodings_nbr,
                              activation=activation,
                              lasso_rate=lasso_rate,
                              sl_lasso_rate=lasso_rate_SL)

Autoencoder Structure-------------------------------------
Model: "DAE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 28)]              0         
                                                                 
 encoder_hidden_layer_full_0  (None, 12)               348       
  (Dense)                                                        
                                                                 
 encoder_hidden_layer_full_1  (None, 12)               156       
  (Dense)                                                        
                                                                 
 encoder_hidden_layer_full_2  (None, 12)               156       
  (Dense)                                                        
                                                                 
 encoding_layer_full (Dense)  (None, 6)                78        
    

In [10]:
#Pre-training
DAE.compile(loss='mean_squared_error',optimizer=optimizers.Adam())
DAE.fit(syn_data_train_scal,syn_data_train_scal,epochs=400,batch_size=batch_size,\
                        shuffle=True
                       )

Epoch 1/400
21/21 [==============================] - 1s 3ms/step - loss: 1.1860
Epoch 2/400
21/21 [==============================] - 0s 2ms/step - loss: 0.8322
Epoch 3/400
21/21 [==============================] - 0s 2ms/step - loss: 0.6932
Epoch 4/400
21/21 [==============================] - 0s 2ms/step - loss: 0.5074
Epoch 5/400
21/21 [==============================] - 0s 2ms/step - loss: 0.3415
Epoch 6/400
21/21 [==============================] - 0s 2ms/step - loss: 0.2560
Epoch 7/400
21/21 [==============================] - 0s 2ms/step - loss: 0.2077
Epoch 8/400
21/21 [==============================] - 0s 2ms/step - loss: 0.1679
Epoch 9/400
21/21 [==============================] - 0s 2ms/step - loss: 0.1319
Epoch 10/400
21/21 [==============================] - 0s 2ms/step - loss: 0.1010
Epoch 11/400
21/21 [==============================] - 0s 2ms/step - loss: 0.0754
Epoch 12/400
21/21 [==============================] - 0s 2ms/step - loss: 0.0557
Epoch 13/400
21/21 [=================

In [11]:
train_prediction_all_feats_scal = DAE.predict(syn_data_train_scal)
train_prediction_all_feats = scaler.inverse_transform(train_prediction_all_feats_scal)
print('Training:')
print('MSE Full Scaled Feats: '+str(mean_squared_error(syn_data_train_scal,train_prediction_all_feats_scal)))
print('MSE Full Real Feats: '+str(mean_squared_error(syn_data_train,train_prediction_all_feats)))

158/158 [==============================] - 0s 1ms/step
Training:
MSE Full Scaled Feats: 3.754895964581434e-08
MSE Full Real Feats: 0.00039153174024163294


In [12]:
test_prediction_all_feats_scal = DAE.predict(syn_data_test_scal)
test_prediction_all_feats = scaler.inverse_transform(test_prediction_all_feats_scal)
print("Testing:")
print('MSE Full Scaled Feats: '+str(mean_squared_error(syn_data_test_scal,test_prediction_all_feats_scal)))
print('MSE Full Real Feats: '+str(mean_squared_error(syn_data_test,test_prediction_all_feats)))

40/40 [==============================] - 0s 1ms/step
Testing:
MSE Full Scaled Feats: 3.9638091215158935e-08
MSE Full Real Feats: 0.0003905023324516571


In [13]:
#Transfer Learning
for lay in range(nbr_hidden_layers+1):
    SelectFeats_Encoder.layers[-(1+lay)].set_weights(FullFeats_Encoder.layers[-(1+lay)].get_weights())

In [14]:
# Feature Selection Training
SelectFeats_Encoder.compile(loss='mean_squared_error',optimizer=optimizers.Adam())
DAE_encodings_train_scal = FullFeats_Encoder.predict(syn_data_train_scal)
SelectFeats_Encoder.fit(syn_data_train_scal,DAE_encodings_train_scal,epochs=600,batch_size=batch_size,\
                        shuffle=True)

158/158 [==============================] - 0s 1ms/step
Epoch 1/600
21/21 [==============================] - 1s 2ms/step - loss: 0.2798
Epoch 2/600
21/21 [==============================] - 0s 2ms/step - loss: 0.2738
Epoch 3/600
21/21 [==============================] - 0s 1ms/step - loss: 0.2681
Epoch 4/600
21/21 [==============================] - 0s 2ms/step - loss: 0.2624
Epoch 5/600
21/21 [==============================] - 0s 2ms/step - loss: 0.2567
Epoch 6/600
21/21 [==============================] - 0s 2ms/step - loss: 0.2509
Epoch 7/600
21/21 [==============================] - 0s 2ms/step - loss: 0.2451
Epoch 8/600
21/21 [==============================] - 0s 2ms/step - loss: 0.2393
Epoch 9/600
21/21 [==============================] - 0s 2ms/step - loss: 0.2335
Epoch 10/600
21/21 [==============================] - 0s 2ms/step - loss: 0.2277
Epoch 11/600
21/21 [==============================] - 0s 1ms/step - loss: 0.2219
Epoch 12/600
21/21 [==============================] - 0s 2ms/st

In [15]:
train_prediction_select_feats_scal = Select_DAE.predict(syn_data_train_scal)
train_prediction_select_feats = scaler.inverse_transform(train_prediction_select_feats_scal)
print('Training:')
print('MSE Select Scaled Feats: '+str(mean_squared_error(syn_data_train_scal,train_prediction_select_feats_scal)))
print('MSE Select Real Feats: '+str(mean_squared_error(syn_data_train,train_prediction_select_feats)))

158/158 [==============================] - 0s 1ms/step
Training:
MSE Select Scaled Feats: 3.2183922408619547e-07
MSE Select Real Feats: 0.0022950268392673352


In [16]:
survived_feats = len(np.where(SelectLayer_output.layers[1].get_weights()[0]>=1e-6)[0])
print('Number of Important Features = '+str(survived_feats))

Number of Important Features = 4


In [17]:
wghts_final = SelectLayer_output.layers[1].get_weights()[0]
print(wghts_final.reshape(wghts_final.shape[0]//4,4))

[[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 1.8338520e-02 1.2449087e-07]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [1.6729781e-02 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [1.5306756e-02 0.0000000e+00 0.0000000e+00 1.7514017e-07]
 [1.7703215e-02 0.0000000e+00 0.0000000e+00 0.0000000e+00]]


In [18]:
test_prediction_select_feats_scal = Select_DAE.predict(syn_data_test_scal)
test_prediction_select_feats = scaler.inverse_transform(test_prediction_select_feats_scal)
print('Testing:')
print('MSE Select Scaled Feats: '+str(mean_squared_error(syn_data_test_scal,test_prediction_select_feats_scal)))
print('MSE Select Real Feats: '+str(mean_squared_error(syn_data_test,test_prediction_select_feats)))

 1/40 [..............................] - ETA: 1s

40/40 [==============================] - 0s 1ms/step
Testing:
MSE Select Scaled Feats: 3.4428461163087496e-07
MSE Select Real Feats: 0.002325149351295041


In [19]:
# Nonlinear Reconstruction
if not LinearData:
    print('Nonlinear Reconstruction Error = ')
    print(mean_squared_error(syn_data_test[:,-12:],test_prediction_select_feats[:,-12:]))

Nonlinear Reconstruction Error = 
0.005213159701823472
